# Monte-Carlo
This section explores the Monte Carlo method for reinforcement learning




## Background Theory

## Code - MC Policy Iteration Experiment

In this part of the code, we are "training" the agent to traverse the gridworld. This is accomplished with generalized policy iteration, using MC method to estimate the actuation value function $Q_\pi(s_t,a_t)$

Note that a customized FrozenLake environment, derived from the base version of gymnasium, is used. In gymnasium FrozenLake, the reset() method always returns the agent to state 0. However, the customized version give it an exploring start, where the agent can start its trajectory in any non-terminal state. This method ensures that each action-state is visited more uniformly. 

In [ ]:
import numpy as np
import gymnasium as gym
import custom_frozenlake  # Make sure this runs


# Initialize the environment
env = gym.make("CustomFrozenLake-v0", map_name="4x4", is_slippery=True, render_mode = "ansi")  # is_slippery=True for stochasticity
obs, info = env.reset()

state = obs; t = 0
# Simulate several steps by following a random policy
for i in range(100):
    action = env.action_space.sample()
    next_state, reward, term, trunc, info = env.step(action)
    print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
    # print(env.render())
    state = next_state; t += 1
    if term or trunc:
        break

In [ ]:
state_size = env.observation_space.n
action_size = env.action_space.n
q_table = np.zeros((state_size,action_size))

In Monte Carlo RL, the agent is supposed to traverse entire episode(s) to observe the trajectory and reward. The return of each state is monitored and used for updating the state-action value $Q(S_t,A_t)$

Need to implement the following functionalities:
1. A `get_one_episode()` function to play out one episode (trajectory) that makes the agent steps through the environment until a termination condition is reached. These trajectories are returned as three lists: 1. states, 2. rewards, and 3. actions
2. A `get_returns()` function to calculate the cumulated return $G_t$ when given a list of state and a list of rewards along a trajectory generated by the function above. 
3. Need a `state_action_values()` array to store the tabular Q-value mapping from state-action $(s_t, a_t)$ to Q-value - $Q(s_t,a_t)$
4. (optional) Need an array to store the occurences of all state-action combinations - $N(s_t,a_t)$. This can however be substituted with a constant $\alpha$




### Question to self
1. How can I specify a policy to control the agent? Thus far `env.step()` randomly chooses the next action for the agent.
- Earlier on, we randomly sampled the action from the action space of the environment. This action was then used in the `env.step()` function to guide the agent through one step. 
- With MC control, we gradually update the Q-value of all the state-action pairs and choose the action according to an epsilon-greedy policy. This policy then replaces the initial random policy and is used for selecting an action for

2. How can I visualize the state-action value on the environment to visualize the policy?

3. When collecting the trajectories for Monte Carlo training (for lack of better terminology), do we use epsilon-greedy in some way or do we use greedy policy?
- Initially, we start with some random policy and turn this into an epsilon-greedy policy to sample/roll out/obtain trajectories.
- The reward from these trajectories are turned into the return at each <s,a> state-action pair. These returns are normalized over the number of occurences of <s,a> and is used as an estimate of the action value function $Q(s,a)$

2. Are there alternatives to epsilon-greedy policy?


In [ ]:
def get_returns(rewards, gamma = 0.9):
    ''' Function to calculate the return of each time step when given a list of rewards 
    
    For each step of th trajectory (of length T):
    - Extract the rewards from that step onward
    - Each step is multiplied by the corresponding gamma ^ index 
        the first reward received from leaving the state is not discounted
        the last reward received from the trajectory is discouned by gamma ^ (T-1)
    - Sum these values together to obtain the return at each step
    '''
    returns = np.zeros(len(rewards))
    
    for step, _ in enumerate(rewards):
        step_reward = rewards[step:]            # reward from the current step onward

        # List of discounted rewards at each time step
        return_val = [gamma ** i * step_reward[i] for i in range(len(step_reward))]
        return_val = sum(return_val)
        
        returns[step] = return_val
    return returns

In [ ]:
def get_one_episode(stochastic_policy):
    ''' This function returns a full trajectory of the agent
    Episode end conditions:
    - the agent falls into the ice or the agent reaches the reward
    - truncated when hitting a time limit

    Inputs:
    - stochastic policy - the stochastic epsilon-greedy policy (array of size action_space x states) 
    '''
    states = []
    rewards = []
    actions = []

    initial_state, _ = env.reset()
    states.append(initial_state)

    term = False
    trunc = False

    # Rollout an entire episode until either terminated or truncated
    while (not term) and (not trunc):
        action = np.random.choice(env.action_space.n,
                                  p = stochastic_policy[:,state])
        next_state, reward, term, trunc, _ = env.step(action)
        
        rewards.append(reward)
        actions.append(action)
        states.append(next_state)


    return states, rewards, actions

In [ ]:
def deterministic_to_stochastic_policy(deterministic_policy, epsilon=0.3):
    ''' Function to convert a deterministic greedy policy to a stochastic epsilon-greedy policy 
    
    A greedy policy selects the action a_t from the state s_t that maximizes Q(s_t,a_t). 
    On the other hand, epsilon-greedy lets the agent explore the environment by following the optimal
    policy (at the time) with 1-epsilon probability, while taking other actions with epsilon probability

    Given an action space of size a, this function converts from a deterministic policy to a stochastic policy
    that follows the optimal action with 1-epsilon probability, and the other (a-1) actions randomly with epsilon probability
    
    Inputs:
    - deterministic_policy - the greedy policy, an array of size 1 x state
    - epsilon - the probability at which the agent take sub-optimal actions randomly to explore
    '''

    num_action = 4          # TO-DO: Improve to avoid hard-coding
    num_state = len(deterministic_policy)

    # Initialize the stochastic policy with all epsilon / 3
    stochastic_policy = np.full((num_action, num_state), epsilon / (num_action - 1))
    for state, action in enumerate(deterministic_policy):
        stochastic_policy[action][state] = 1 - epsilon

    return stochastic_policy


In [ ]:
def MC_policy_evaluation(stochastic_policy, n_episodes = 100, gamma = 0.9):
    ''' This function samples trajectories under a specified stochastic policy to approximate the value function 
    
    (Value function evaluation from multiple trajectories)

    Inputs:
    - stochastic_policy - the policy that the agent follows to roll out trajectories. A matrix of size num_actions x num_states
    - n_episodes - the number of trajectories sampled over which we find the average return G_t (to approximate the true action value function)
    
    Outputs:
    - policy - the deterministic greedy policy (the action in each state that maximizes the action value function)
    - state_action_values - array of size (n_actions x n_states)
    '''

    n_actions = stochastic_policy.shape[0]      # size of action space
    n_states = stochastic_policy.shape[1]       # size of state space

    state_action_return_total = np.zeros((n_actions, n_states))
    state_action_count = np.zeros((n_actions, n_states))

    for episode in range(n_episodes):
        # Get one full episode
        states, rewards, actions = get_one_episode(stochastic_policy)

        # Calculate the return
        returns = get_returns(rewards, gamma)

        # Update the total return for each state-action pair in the trajectory
        for idx, _ in enumerate(states[:-1]):       # iterate through the trajectory (terminal state excluded)
            state_action_return_total[actions[idx],states[idx]] += returns[idx]
            state_action_count[actions[idx],states[idx]] += 1

    # Update the state-action value function by dividing the total state
    state_action_values = state_action_return_total / (state_action_count + 0.0001)

    # Deterministic greedy policy
    policy = np.argmax(state_action_values, axis=0)

    return policy, state_action_values

Now that we have a function to perform value function evaluation, we need to somehow incorporate this improved evaluation into the improvement of the control policy. The output of policy evaluation are: 1. the state-action values $Q(s,a)$ $\forall s \in \mathcal{S}, \, a \in \mathcal{A}$, and 2. the deterministic greedy policy.

We will use this new deterministic greedy policy, first converted to an epsilon-greedy policy, for building out trajectories for the next policy evaluation process



In [ ]:
# Initial random deterministic policy
policy = np.random.choice(size = env.observation_space.n,
                          a = np.arange(0,4,1))
gamma = 0.8                 # discount factor in calculating the return at each step
epsilon = 0.3               # probability of exploratory moves in epsilon-greedy

n_episodes = 100            # number of trajectories considered when evaluating the policy
n_policy_updates = 200   # number of policy improvement iterations

all_policies = []
count = 0

for policy_update in range(n_policy_updates):
    # Convert deterministic greedy policy to stochastic epsilon-greedy
    stochastic_policy = deterministic_to_stochastic_policy(policy,epsilon = 0.6)

    # MC policy evaluation
    policy, state_action_values = MC_policy_evaluation(stochastic_policy, 
                                                       n_episodes = 5000, 
                                                       gamma = gamma)

    if (policy_update % 50 == 0) or (policy_update == n_policy_updates - 1):

        all_policies.append(policy)
        count += 1
        print(f"Updated policy {count:2d} - {policy}")



In [ ]:
print(state_action_values)
print(np.argmax(state_action_values, axis = 0))

Now we can simulate the trajectory of the agent according to the stochastic epsilon-greedy policy generated above.


In [ ]:
env_test = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode = "human")  # is_slippery=True for stochasticity
env_test.reset()


for i in range(10):

    env_test.reset()
    state = 0; t = 0
    stochastic_policy = deterministic_to_stochastic_policy(policy)
    term = False; trunc = False

    while (not term) and (not trunc):
        action = np.random.choice(a = env_test.action_space.n, 
                                p = stochastic_policy[:,state])
        next_state, reward, term, trunc, info = env_test.step(action)
        # print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
        # print(env.render())
        state = next_state; t += 1

We can also plot the value function at the various state-action pairs in a 3D plot. In this plot, the x-y axes are the state and action, creating a mesh grid. The z-axis is then the value function at each vertex of the meshgrid

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib widget

actions = np.arange(state_action_values.shape[0])
states = np.arange(state_action_values.shape[1])

A, S = np.meshgrid(actions,states)
Q = state_action_values[A,S]

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(A, S, Q, cmap='viridis')
ax.set_xlabel('X (row)')
ax.set_ylabel('Y (col)')
ax.set_zlabel('Value V(s)')
ax.set_xticks(actions)
ax.set_yticks(states)


plt.title("Action-State Value Function Surface")
plt.show()

# TD(0) Learning

In temporal difference learning for control (which is used for **policy evaluation**), the agent traverse the environment step-by-step to bootstrap the action value function. As the agent take one step from a state $S_t$, it will receive a reward $R_{t+1}$ and reach a state $S{t+1}$. From this state, the agent takes another action $A_{t+1}$, which has a corresponding action value of $Q(S_{t+1},A_{t+1})$. These two values are used for backing up and updating the action value at time t, $Q(S_t,A_t)$ as 

$$ \begin{aligned} 
Q(S_t,A_t) &\leftarrow Q(S_t,A_t) + \alpha \Big(R(S_t,A_t) + \gamma Q\left(S_{t+1},A_{t+1}\right) - Q\left(S_t, A_t\right)\Big) \\
Q(S_t,A_t) &\leftarrow (1-\alpha) \cdot Q(S_t,A_t) + \alpha \Big(R(S_t,A_t) + \gamma Q\left(S_{t+1},A_{t+1}\right) \Big)
\end{aligned}  $$ 

One key difference between Monte Carlo and Temporal Difference Learning is the frequency of action-value updates. While MC waits for an entire episode to simulate, TD can update the action-value function as the agent steps through the environment. One can see that when given an arbitrary Q function, and thus an arbitrary policy, the agent slowly updates the state near the end first. The known action value of those states (which is zero) propagates backward after multiple episodes.

The process works slowly backward from the terminal states with known rewards, propagating outward to neighboring states before reaching the starting states


## Question to self
1. How does the agent take the action $A_t$ from state $S_t$ or action $A_{t+1}$ from state $S_{t+1}$? 
- Since temporal difference learning is used for policy evaluation in control, I think that we are using TD to evaluate a given policy (initially a random epsilon-greedy policy)
- We sample the optimal action in a given state according to this policy, with some chance of exploring other actions as well. 

2. With MC, it makes sense to incorporate the average of $R(S_t,A_t)$ from multiple trajectories to derive the action value function. However, how does running multiple trajectories for TD help? How does the knowledge gained from one trajectory carries to the next?
- Running multiple trajectories help propagate the value backward from terminal states with known rewards, thus approaching closer to the true value function.
- However, it has been proven that even with one episode of policy evaluation, we can still get a policy that is never worse than the previous policy, or $\pi'(s) \ge \pi(s)$

3. Does exploring start assist or sabotage the learning process?

We will reuse three functions defined earlier: `get_returns()`, `get_one_episode()`, and `deterministic_to_stochastic_policy()`. However, we will rewrite the update_MC_policy() to update according to TD(0)

In [ ]:
# Custom, simplified frozen lake drawing function to visualize the policy
from frozen_lake_plot import plot_grid

# Initialize the environment
env = gym.make("CustomFrozenLake-v0", map_name="4x4", is_slippery=True, render_mode = "ansi")  # is_slippery=True for stochasticity
obs, info = env.reset()

state = obs; t = 0
# Simulate several steps by following a random policy
for i in range(100):
    action = env.action_space.sample()
    next_state, reward, term, trunc, info = env.step(action)
    print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
    # print(env.render())
    state = next_state; t += 1
    if term or trunc:
        break

In [ ]:
def TD_policy_evaluation(stochastic_policy, n_episodes = 100, gamma = 0.9, alpha = 0.5):
    ''' This function samples trajectories under a specified stochastic policy to approximate the value function according to TD updates 
    Value function approximation is done over a specified number of trajectories 
    

    Inputs:
    - stochastic_policy - the policy that the agent follows to roll out trajectories. A matrix of size num_actions x num_states
    - n_episodes - the number of trajectories sampled over which we find the average return G_t (to approximate the true action value function)
    - gamma - the discount factor for reward signals
    - alpha - learning rate/update rate

    Outputs:
    - policy - the deterministic greedy policy (the action in each state that maximizes the action value function)
    - state_action_values - array of size (n_actions x n_states)
    '''

    n_actions = stochastic_policy.shape[0]      # size of action space
    n_states = stochastic_policy.shape[1]       # size of state space

    state_action_values = np.zeros((n_actions, n_states))

    for episode in range(n_episodes):
        start_state, info = env.reset()             # Reset the agent to an arbitrary start state (non-terminal)
        state = start_state
        term, trunc = False, False

        # Choose an initial action from this initial state according to the stochastic policy
        action = np.random.choice(a = range(n_actions), 
                                  p = stochastic_policy[:,state])
        
        # Step through the environment to update the action value function
        while (not term) and (not trunc):
            # Carry out the action previously chosen, then sample another action A_{t+1} when reaching state S_{t+1}
            next_state, reward, term, trunc, info = env.step(action)
            next_action = np.random.choice(a = range(n_actions), 
                                           p = stochastic_policy[:,next_state])
            
            # Update the action value function
            state_action_values[action,state] += alpha * (reward + gamma * state_action_values[next_action, next_state] - state_action_values[action,state])

            state = next_state
            action = next_action

        policy = np.argmax(state_action_values, axis=0)    
    return policy, state_action_values

In [ ]:
def success_metric(policy, n_episodes_test = 5000):

    ''' Assess the success rate of the current policy in terms of success rate and the time to reach the goal'''
    env_test = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode = "ansi")  # is_slippery=True for stochasticity
    env_test.reset()

    success_count = 0           # Number of successful episode
    step_count = 0              # Number of steps in a successful episode

    for i in range(n_episodes_test):

        env_test.reset()
        state = 0; t = 0
        stochastic_policy = deterministic_to_stochastic_policy(policy, epsilon=0)

        term = False; trunc = False
        states = []; states.append(state)

        while (not term) and (not trunc):
            
            action = np.random.choice(a = env_test.action_space.n, 
                                    p = stochastic_policy[:,state])
            next_state, reward, term, trunc, info = env_test.step(action)
            # print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
            # print(env.render())

            states.append(next_state)

            state = next_state; t += 1

            if next_state == 15: 
                success_count += 1 
                step_count += len(states)

    success_rate = success_count / n_episodes_test * 100
    step_average_per_eps = np.round(step_count / (success_count+0.0001))

    return success_rate, step_average_per_eps

In [ ]:
def TD_policy_improvement(policy, n_policy_eval, n_policy_updates,
                          gamma, epsilon, eps_decay, eps_decay_rate, alpha):
    
    ''' Policy improvement function that incorporates SARSA TD policy evaluation 
    
    Inputs:
    - policy - an array of size (1 x state_space) representing the deterministic policy with one chosen action in each state
    - n_policy_eval - the number of episodes simulated to evaluate the current policy Q approx q_pi
    - n_policy_updates - the number of iterations to update the policy
    - gamma - the discount factor for calculating the return from the rewards of a trajectory
    - epsilon - the exploration rate for turning a deterministic greedy policy to an stochastic epsilon-greedy policy
    - eps_decay - the epsilon decay factor
    - eps_decay_rate - the frequency of epsilon decay (every eps_decay_rate amount of policy improvement, epsilon is multiplied by the eps_decay factor)
    - alpha - the learning/updating rate of the Q-value funciton
    '''
    
    all_policies = []           # Record the policies
    success_rates = []
    step_average_per_eps_list = []

    for policy_update in range(n_policy_updates):
        # Convert deterministic greedy policy to stochastic epsilon-greedy
        stochastic_policy = deterministic_to_stochastic_policy(policy, epsilon)

        # TD policy evaluation
        policy, state_action_values = TD_policy_evaluation(stochastic_policy, 
                                                        n_episodes = n_policy_eval, 
                                                        gamma = gamma,
                                                        alpha = alpha)
        
        # Step decay of epsilon
        if (policy_update % eps_decay_rate == 0) and (policy_update != 0):           # Half the exploration rate every few policy updates
            epsilon = epsilon * eps_decay
            print(f"Current epsilon value - {epsilon}")

        # # Exponential decay of epsilon
        # epsilon *= eps_decay ** (policy_update/eps_decay_rate)

        # Visualize the policy update and track the performance once in a while
        if (policy_update % 50 == 0) or (policy_update == n_policy_updates - 1):
            all_policies.append(policy)
            print(f"Updated policy {policy_update:2d} {policy} \n {policy.reshape(4,-1)}")
            plot_grid(4, state_action_values, f"Policy {policy_update:2d}")

            # Simulate several test runs with the current policy to obtain success rate and average time taken
            success_rate, step_average_per_eps = success_metric(policy)

            success_rates.append(success_rate)
            step_average_per_eps_list.append(step_average_per_eps)

    return policy, all_policies, state_action_values, success_rates, step_average_per_eps_list

In [ ]:
# policy = np.random.choice(size = env.observation_space.n, a = np.arange(0,4,1))
policy = np.zeros(16, dtype=int)

gamma = 0.8                 # discount factor in calculating the return at each step
epsilon = 0.6               # probability of exploratory moves in epsilon-greedy
alpha = 0.05

n_policy_eval = 100         # number of trajectories considered when evaluating the policy
n_policy_updates = 2500     # number of policy improvement iterations

eps_decay = 0.25
eps_decay_rate = 500

all_policies = []
count = 0

policy, policy_history, state_action_values, success_rates, step_average_per_eps_list = TD_policy_improvement(policy, n_policy_eval, n_policy_updates, 
                                                                                                              gamma, epsilon, eps_decay, eps_decay_rate, alpha)

In [ ]:
for idx, item in enumerate(zip(success_rates, step_average_per_eps_list)):
    print(f"At policy {idx*50:4d} - Success rate of {item[0]:5.2f}% and takes {item[1]:3.0f} steps")

In [ ]:
# Visualize the history of success rate
import matplotlib.pyplot as plt

iter = np.zeros(len(success_rates))
success_rate_list = np.zeros(len(success_rates))
step_count_list = np.zeros(len(step_average_per_eps_list))


for idx, item in enumerate(zip(success_rates, step_average_per_eps_list)):
    iter[idx] = idx * 50
    success_rate_list[idx] = item[0]
    step_count_list[idx] = item[1]

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))  # 2 rows, 1 column

# Plot y1 vs x on the first subplot
ax1.plot(iter, success_rate_list, marker='o', color='blue')
ax1.set_title('Success Rate vs Policy Update Iteration')
ax1.set_xlabel('x')
ax1.set_ylabel('y1')

# Plot y2 vs x on the second subplot
ax2.plot(iter, step_count_list, marker='s', color='green')
ax2.set_title('Number of steps vs Policy Update Iteration')
ax2.set_xlabel('x')
ax2.set_ylabel('y2')

# Adjust layout
plt.tight_layout()
plt.show()

Test out the policy above in a simulation

In [ ]:
env_test = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode = "human")  # is_slippery=True for stochasticity
env_test.reset()

success_count = 0           # Number of successful episode
step_count = 0              # Number of steps in a successful episode
n_episodes_test = 5

# Benchmark against a random deterministic policy
random_policy = np.random.choice(size = env.observation_space.n, a = np.arange(0,4,1))


for i in range(n_episodes_test):

    env_test.reset()
    state = 0; t = 0
    stochastic_policy = deterministic_to_stochastic_policy(policy, epsilon=0)

    term = False; trunc = False
    states = []; states.append(state)
    actions = []

    while (not term) and (not trunc):
        
        action = np.random.choice(a = env_test.action_space.n, 
                                p = stochastic_policy[:,state])
        next_state, reward, term, trunc, info = env_test.step(action)
        # print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
        # print(env.render())

        states.append(next_state)
        actions.append(action)

        state = next_state; t += 1

        if next_state == 15: 
            success_count += 1 
            step_count += len(states)

success_rate = success_count / n_episodes_test * 100
step_average_per_eps = np.round(step_count / (success_count+0.0001))
print(f"Success rate of {success_rate:.5f}% \n Average number of step required - {step_average_per_eps:3f}")

Visualize the state-action values on a 3D grid

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib widget

actions = np.arange(state_action_values.shape[0])
states = np.arange(state_action_values.shape[1])

A, S = np.meshgrid(actions,states)
Q = state_action_values[A,S]

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(A, S, Q, cmap='viridis')
ax.set_xlabel('X (row)')
ax.set_ylabel('Y (col)')
ax.set_zlabel('Value V(s)')
ax.set_xticks(actions)
ax.set_yticks(states)


plt.title("Action-State Value Function Surface")
plt.show()

# TD($\lambda$) Learning

# REINFORCE algorithm

This experiment explores the policy gradient algorithm REINFORCE in the FrozenLake environment with discrete action space.

TODO:
- Policy network class
- Initialize the environment, network, and optimizer

- Training loop through num_episode_train


In [ ]:
import gymnasium as gym
import custom_frozenlake
from frozen_lake_plot import plot_grid

import torch
from torch import nn
import torch.optim as optim
from torch.distributions import Categorical
from torchsummary import summary 

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    device_idx = torch.cuda.current_device()
    print(f"Number of GPU(s): {torch.cuda.device_count()}")
    print(f"Current device: {device_idx}|{torch.cuda.get_device_name(device_idx)}")

## Setup and Hyperparameter Definition

In [ ]:
LR = 1e-4
GAMMA = 0.95
N_EPISODE_TRAIN = 10000
N_EPISODE_TESTING = 10           # for testing functionalities, not actual model testing
CUDA_ENABLED = True

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers=[64,32]):
        ''' Initialize the model and create a list of layers '''
        super().__init__()

        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim

        for layer_size in hidden_layers:
            self.layers.append(nn.Linear(input_dim, layer_size))
            input_dim = layer_size

        self.layers.append(nn.Linear(input_dim, self.output_dim))
        if CUDA_ENABLED: self.cuda()

    def forward(self, input):
        ''' input is a one hot encoded state vector '''
        # If tensor of state_idx, convert to 
        for layer in self.layers[:-1]:
            input = torch.relu(layer(input))
        logits = self.layers[-1](input)
        probs = torch.softmax(logits,dim=-1)
        return probs

In [ ]:
policy_net_test = PolicyNet(16,4)
print(next(policy_net_test.parameters()).is_cuda)
summary(policy_net_test, (16,))

In [ ]:
env = gym.make("CustomFrozenLake-v0")
obs_space = env.observation_space.n
act_space = env.action_space.n

# Simulate several episode in this environment
for i in range(5):
    print(f"Episode {i+1}")
    obs, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_obs, reward, term, trunc, _ = env.step(action)
        print(f"{obs:2d} | {action:2d} | {reward:2.1f} | {next_obs:2d}")
        obs = next_obs
        done = term or trunc

In [ ]:
def one_hot_encode(state_space, state_idx):
    # Convert scalar state value to one hot encoding
    one_hot_encoded_state = torch.zeros(state_space, device='cuda' if CUDA_ENABLED else 'cpu')
    one_hot_encoded_state[state_idx] = 1.0
    return one_hot_encoded_state

In [ ]:
def plot_prob_grid(grid_size, policy, title):
    prob_value_array = []
    for state_idx in range(grid_size * grid_size):
        prob_value_array.append(policy(one_hot_encode(obs_space, state_idx)))
    
    prob_value_array_np = torch.stack(prob_value_array)
    prob_value_array_np = prob_value_array_np.cpu().detach().numpy().T
    plot_grid(grid_size, prob_value_array_np, title)

In [ ]:
def get_returns(rewards, gamma = 0.9):
    ''' Function to calculate the return of each time step when given a list of rewards 
    
    Parameters:
    ----------
    rewards : list
        a list of rewards achieved throughout the agent's trajectory
    gamma   : float
        the discount factor
    
    Returns:
    ----------
    returns : list
        a list of returns G_t at each step of the trajectory
        
    For each step of th trajectory (of length T):
    - Extract the rewards from that step onward
    - Each step is multiplied by the corresponding gamma ^ index 
        the first reward received from leaving the state is not discounted
        the last reward received from the trajectory is discouned by gamma ^ (T-1)
    - Sum these values together to obtain the return at each step
    '''
    returns = np.zeros(len(rewards))
    
    for step, _ in enumerate(rewards):
        step_reward = rewards[step:]            # reward from the current step onward

        # List of discounted rewards at each time step
        return_val = [gamma ** i * step_reward[i] for i in range(len(step_reward))]
        return_val = sum(return_val)
        
        returns[step] = return_val
    return returns

## Training the policy net with REINFORCE (Monte Carlo)

In [ ]:
policy_net = PolicyNet(obs_space, act_space, [64,32])
optimizer = optim.Adam(policy_net.parameters(), lr=LR)
value_history = np.zeros(N_EPISODE_TRAIN)
loss_history = np.zeros(N_EPISODE_TRAIN)

# summary(policy_net, (obs_space, ), device = 'cuda' if CUDA_ENABLED else 'cpu')
NUM_RUN = N_EPISODE_TRAIN

for i in range(NUM_RUN):  
    obs, _ = env.reset()
    done = False
    log_probs = []          # Record a history of lob probability of the policy at each step
    rewards = []            # Record the reward throughtout the episode

    while not done: # Follow a policy network until terminated
        # Select action according to the current policy network
        obs_onehot = one_hot_encode(obs_space, obs)
        # with torch.no_grad():
        action_probs = policy_net(obs_onehot)
        # print(action_probs)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()

        next_obs, reward, term, trunc, _ = env.step(action.item())
        # print(f"{obs:2d} | {action:2d} | {reward:2.1f} | {next_obs:2d}")
        rewards.append(reward)
        obs = next_obs
        done = term or trunc
    
        log_probs.append(action_dist.log_prob(action))  # this is not wrapped in torch.no_grad() since it is used for loss calculation

    returns = get_returns(rewards, gamma = GAMMA)
    value_history[i] = returns[0]

    log_freq = 100
    if i == 0 or i % log_freq == (log_freq - 1):
        print(f"Training Episode {i+1:4d}/{NUM_RUN} \n- Rewards {rewards} \n- Returns {returns} \n- Return R_1 {returns[0]:3.3f}", end = "\r")
        plot_prob_grid(4, policy_net, title = f"FrozenLake episode {i+1:4d}")
        
    
    # Normalize the returns to reduce variance
    # returns = (returns - returns.mean()) / (returns.std() + 1e-8)
    returns = torch.tensor(returns, dtype = torch.float32, device='cuda' if CUDA_ENABLED else 'cpu')

    loss = -torch.sum(torch.stack(log_probs) * returns)
    loss_history[i] = loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.figure(figsize=(20,8))
plt.plot(value_history)

## Simulate episodes with the current policy net

In [ ]:
env_test = gym.make("FrozenLake-v1")
obs_space = env_test.observation_space.n

success_count = 0
N_EPISODE_TEST = 10000

for epsidode in range(N_EPISODE_TEST):
    obs, _ = env_test.reset()
    done = False
    while not done:
        obs = one_hot_encode(obs_space, obs)
        action_probs = policy_net(obs)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()

        obs, reward, term, trunc, _ = env_test.step(action.item())
        if obs == 15:
            success_count += 1
        done = term or trunc

print(f"Success rate of {success_count/N_EPISODE_TEST*100:4.2f}%")